In [ ]:
import csv
import pandas as pd
import string
# import requests
from xml.dom import minidom
import nltk
from bs4 import BeautifulSoup
from pathlib import Path
dataset_dir = Path.cwd() / 'datasets/stackexchange'

In [ ]:
import xml.etree.ElementTree as ET

def xml2df(path):
    tree = ET.parse(path)
    root = tree.getroot()
    data=[]
    for child in root:
        #print(child.attrib)
        #df = pd.Series(child.attrib).to_frame()
        data.append(child.attrib)
    df = pd.DataFrame(data)
    return df

In [ ]:
# Posts.xml 
# PostTypeId = 1 Qeustion, Id: QuestionId, Tags: Category, OwnerUserId: AskerId(UserId), Tags: Category,
# PostTypeId = 2 Answer, Id: AnswerId, ParentId： QuestionId, Score: Helpfulness, CreationDate: Timestamp, 
# OwnerUserId: AnswererId(UserId)
# Users.xml Reputation: Reputation, DsiplayName: Username, Id: UserId
# Tags.xml TagName: Category

In [ ]:
from pathlib import Path
abs_data_dir = Path.cwd() / 'csv_files/stackexchange/content'
# data_dir = Path.home() / 'GenderPerformance/datasets/yelp'
dataset_dir = Path.cwd() / 'csv_files/stackexchange'

def create_data(category):
    df_post= xml2df(abs_data_dir / 'Posts.xml')
    df_user= xml2df(abs_data_dir / 'Users.xml')
    
    print("Posts: ",df_post.shape)
    df_post = df_post[['OwnerUserId', 'Body', 'PostTypeId', 'CreationDate', 'Score']]
    df_post = df_post[df_post['PostTypeId'].apply(lambda x: x == '2')]
    df_post.rename(columns={'OwnerUserId': 'UserId'}, inplace=True)

    print("Users: ", df_user.shape)
    df_user = df_user[['Id', 'DisplayName', 'Reputation']]
    df_user.rename(columns={'Id': 'UserId'}, inplace=True)
    data = pd.merge(df_post, df_user, on='UserId', how='left')
    data.dropna(inplace=True)
    print("Combined frame shape", data.shape)
#     data.drop(['UserId'], axis=1, inplace=True)
    data.rename(columns={'DisplayName':'Name', 'Body': 'AnswerText'}, inplace=True)
#     data = data.reindex(columns=['Name', 'AnswerText'])

    data['AnswerText'] = data['AnswerText'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text().replace('|', '').lower())
    data['AnswerText'] = data['AnswerText'].apply(lambda x: " ".join(x.split()))
#     soup = BeautifulSoup(data.at[0,'Post'], 'html.parser')
#     print(soup.get_text())
    data.to_csv(dataset_dir /  (category + ".csv"), sep='|', encoding='utf-8', header=False, index=False, na_rep=string)
    print("Write complete for post: ", category)
    return data

In [ ]:
from pathlib import Path

dataset_dir = Path.cwd() / 'datasets/stackexchange'


# def create_data(category):
df_comments = xml2df(dataset_dir / 'Comments.xml')
df_users = xml2df(dataset_dir / 'Users.xml')

print("Posts: ",df_comments.shape)
df_comments = df_comments[['UserId', 'Text', 'CreationDate', 'Score']]

print("Users: ", df_users.shape)
df_users = df_users[['Id', 'DisplayName', 'Reputation']]
df_users.rename(columns={'Id': 'UserId', 'DisplayName': 'UserName'}, inplace=True)
merged_df = pd.merge(df_comments, df_users, on='UserId', how='left')
merged_df.dropna(inplace=True)
print("Combined frame shape", merged_df.shape)


# merged_df['Text'] = merged_df['Text'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text().replace('|', '').lower())
# merged_df['Text'] = merged_df['Text'].apply(lambda x: " ".join(x.split()))

merged_df.to_csv(dataset_dir /  "stackoverflow_full_dataset.csv", sep='|', encoding='utf-8', header=False, index=False, na_rep=string)
print("Write complete for post: ", category)


In [ ]:
merged_df.to_csv(dataset_dir /  "stackoverflow_full_dataset.csv", sep='|', encoding='utf-8', index=False, na_rep=string)

In [ ]:
# df = create_data('stackoverflow_full_dataset')
merged_sampled_df = merged_df.sample(6000000)

In [ ]:
merged_sampled_df.to_csv(dataset_dir /  "stackoverflow_sampled_dataset.csv", sep='|', encoding='utf-8', index=False, na_rep=string)

In [ ]:
merged_sampled_df = merged_sampled_df.reset_index(drop=True)

In [ ]:
#UnicodeEncodeError: 'utf-8' codec can't encode character '\udbff' in position 789: surrogates not allowed

In [ ]:
def cleanser(text):
    post = ''
    try:
        post = text.encode("utf-8")
    except UnicodeEncodeError as e:
        if e.reason == 'surrogates not allowed':
            post =  text.encode('utf-8', "backslashreplace").decode('utf-8')
    
    return post

In [ ]:
df['AnswerText'] = df['AnswerText'].apply(lambda x: cleanser(x))

In [ ]:
df.to_csv(dataset_dir / 'so_train_full_data.csv' , sep='|', header=False, index=False, na_rep=string)

In [ ]:
def clean(text):
    post =''
    try:
        post = text.decode('utf-8')
    except AttributeError:
        post =  text
    return post

In [ ]:
df['AnswerText'] = df['AnswerText'].apply(lambda x: clean(x))

In [ ]:
df.to_csv(dataset_dir / 'so_train_full_data.csv', sep='|', encoding='utf-8', header=False, index=False, na_rep=string)

# gender guesser 

In [ ]:
df = pd.read_csv(dataset_dir / 'stackoverflow_sampled_dataset.csv', sep='|', encoding='utf8', name=[]'UserId', 'Text', 'CreationDate', 'Score', 'UserName', 'Reputation'])
df = pd.read_csv(dataset_dir / 'stackoverflow_full_dataset.csv', sep='|', encoding='utf8', )

In [ ]:
df.columns = ['UserId', 'Text', 'CreationDate', 'Score', 'UserName', 'Reputation']

In [ ]:
df = merged_sampled_df

In [ ]:
import gender_guesser.detector as gender

d = gender.Detector(case_sensitive=False)
translator = str.maketrans('', '', string.punctuation)

def get_gender(name):
    try:
        n = name.lower()
        if ('mom' in n) or ('girl' in n) or ('angel' in n) or ('mum' in n) or ('mother' in n) or ('woman' in n):
            return 'female'
        if ('boy' in n) or ('dude' in n):
            return 'male'
        temp = name.translate(translator).split()
    except:
        return 'unknown'
    
    if len(temp)>0:
        first_name = temp[0]
        first_name = ''.join([i for i in first_name if not i.isdigit()])
        gender = d.get_gender(first_name,'usa')
        if(gender=='unknown'):
            first_name= first_name[:-1]
            gender = d.get_gender(first_name,'usa')
        return gender
    else:
        return 'unknown'

In [ ]:
df['Gender'] = df['UserName'].apply(lambda x:get_gender(x))

In [ ]:
df.Gender.value_counts()

In [ ]:
df.head()

In [ ]:
# disclosed_flag = df.Gender.isin(['mostly_male', 'male', 'mostly_female', 'female'])
disclosed_flag = df.Gender.isin(['male', 'female'])
undisclosed_flag = ~disclosed_flag

In [ ]:
disclosed_gender_df =  df[disclosed_flag]
undisclosed_gender_df = df[undisclosed_flag]

In [ ]:
import os.path
import pandas as pd
if not os.path.exists(dataset_dir / 'disclosed_dataset.csv'):
    print('saving disclosed dataset to csv')
    disclosed_gender_df.to_csv(dataset_dir / 'disclosed_dataset.csv', index=False)
else:
    disclosed_gender_df = pd.read_csv(dataset_dir / 'disclosed_dataset.csv')

if not os.path.exists(dataset_dir / 'undisclosed_dataset.csv'):
    print('saving undisclosed dataset to csv')
    undisclosed_gender_df.to_csv(dataset_dir / 'undisclosed_dataset.csv', index=False)
else:
    undisclosed_gender_df = pd.read_csv(dataset_dir / 'undisclosed_dataset.csv')

In [ ]:
dataset_dir = Path.cwd() / 'datasets/stackexchange/'

In [ ]:
# mapping = {'female' : 1, 'male' : 0, 'mostly_female': 1, 'mostly_male': 0}
mapping = {'female' : 1, 'male' : 0}
disclosed_dataset_df = disclosed_gender_df[['UserName', 'Text', 'Gender']]
disclosed_dataset_df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
disclosed_dataset_df.replace({'Gender': mapping}, inplace=True)

undisclosed_dataset_df = undisclosed_gender_df[['UserName', 'Text', 'Gender']]
undisclosed_dataset_df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
# undisclosed_dataset_df.to_csv(dataset_dir / 'undisclosed_id_text_gender.csv', index=False, header=False)

# plot data distribution

In [ ]:
DF_shape = disclosed_dataset_df.loc[disclosed_dataset_df['Gender'] == 1].shape
DM_shape = disclosed_dataset_df.loc[disclosed_dataset_df['Gender'] == 0].shape
UNDIS_shape = undisclosed_dataset_df.shape
total_size = disclosed_dataset_df.shape[0] + UNDIS_shape[0]

In [ ]:
total_size = disclosed_dataset_df.shape[0] + UNDIS_shape[0]

In [ ]:
print(total_size, disclosed_dataset_df.shape[0], UNDIS_shape[0], DM_shape[0], DF_shape[0])

In [ ]:
import matplotlib.pyplot as plt
labels = 'SM', 'SW', 'Performing'
sizes = [DM_shape[0]/total_size, DF_shape[0]/total_size, UNDIS_shape[0]/total_size]
explode = (0, 0, 0)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

# generate training/test datasets

In [ ]:
from sklearn.model_selection import train_test_split

# dataset_df = pd.read_csv('train_dataset.csv')
if 'disclosed_dataset_df' in locals():
    # 80/20 train/test
    train_df, test_df = train_test_split(disclosed_dataset_df, test_size=0.2)
    # 80/20 train/validation
    train_df, validation_df = train_test_split(train_df, test_size=0.2)
else:
    disclosed_dataset_df = pd.read_csv(dataset_dir / 'disclosed_dataset.csv')
    # 80/20 train/test
    train_df, test_df = train_test_split(disclosed_dataset_df, test_size=0.2)
    # 80/20 train/validation
    train_df, validation_df = train_test_split(train_df, test_size=0.2)

In [ ]:
# extracting related attributes for training, validation and test
train_gender_text_df = train_df[['Gender', 'Text']]
validation_gender_text_df = validation_df[['Gender', 'Text']]
test_name_text_gender_df = test_df[['UserName', 'Text', 'Gender']]

train_gender_text_df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
train_gender_text_df.to_csv(dataset_dir / 'training_gender_text.csv', index=False, header=False)

validation_gender_text_df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
validation_gender_text_df.to_csv(dataset_dir / 'validation_gender_text.csv', index=False, header=False)

test_name_text_gender_df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
test_name_text_gender_df.to_csv(dataset_dir / 'test_name_text_gender.csv', index=False, header=False)

In [ ]:
import pandas as pd
undisclosed_dataset_df = pd.read_csv(dataset_dir / 'undisclosed_dataset.csv')
undisclosed_dataset_df.head()

In [ ]:
undisclosed_gender_df = undisclosed_dataset_df[['UserId', 'Text', 'Gender']]
undisclosed_gender_df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
undisclosed_gender_df.to_csv(dataset_dir / 'undisclosed_id_text_gender.csv', index=False, header=False)

In [ ]:
undisclosed_dataset_df.head()

# plot data distribution

In [ ]:
sampled_df = pd.read_csv(dataset_dir / 'stackoverflow_sampled_dataset.csv', sep='|', encoding='utf8', names=['UserId', 'Text', 'CreationDate', 'Score', 'UserName', 'Reputation'])

In [ ]:
sampled_df['Gender'] = sampled_df['UserName'].apply(lambda x:get_gender(x))

In [ ]:
sampled_df.shape

In [ ]:
disclosed_flag = sampled_df.Gender.isin(['male', 'female'])
undisclosed_flag = ~disclosed_flag

In [ ]:
disclosed_gender_sampled_df =  sampled_df[disclosed_flag]
undisclosed_gender_sampled_df = sampled_df[undisclosed_flag]

In [ ]:
mapping = {'female' : 1, 'male' : 0}
disclosed_dataset_sampled_df = disclosed_gender_sampled_df[['UserName', 'Text', 'Gender']]
disclosed_dataset_sampled_df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
disclosed_dataset_sampled_df.replace({'Gender': mapping}, inplace=True)

undisclosed_dataset_sampled_df = undisclosed_gender_sampled_df[['UserName', 'Text', 'Gender']]
undisclosed_dataset_sampled_df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)

In [ ]:
DF_shape = disclosed_dataset_sampled_df.loc[disclosed_dataset_sampled_df['Gender'] == 1].shape
DM_shape = disclosed_dataset_sampled_df.loc[disclosed_dataset_sampled_df['Gender'] == 0].shape
UNDIS_shape = undisclosed_dataset_sampled_df.shape

In [ ]:
total_size = disclosed_gender_sampled_df.shape[0] + UNDIS_shape[0]

In [ ]:
print(total_size, disclosed_gender_sampled_df.shape[0], UNDIS_shape[0], DM_shape[0], DF_shape[0])

In [ ]:
import matplotlib.pyplot as plt
labels = 'DM', 'DF', 'UNDISCLOSED'
sizes = [DM_shape[0]/total_size, DF_shape[0]/total_size, UNDIS_shape[0]/total_size]
explode = (0, 0, 0)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()